In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from matplotlib.backends.backend_pdf import PdfPages

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor


# -----------------------------
# 1) ساخت دیتای شبیه‌سازی‌شده
# -----------------------------
def make_time_index(days=30, start="2025-08-01"):
    # freq="H" deprecated -> "h"
    return pd.date_range(start=start, periods=24 * days, freq="h")


def make_temperature(idx, base=34, daily_amp=4, noise_std=1.2, seed=42):
    rng = np.random.default_rng(seed)
    hour = idx.hour.values
    temp = base + daily_amp * np.sin(2 * np.pi * (hour - 14) / 24) + rng.normal(
        0, noise_std, size=len(idx)
    )
    return temp


def base_profile(class_name):
    h = np.arange(24)
    if class_name == "res":  # خانگی: پیک عصر
        prof = 0.6 + 0.25 * np.sin(2 * np.pi * (h - 18) / 24) + 0.10 * np.sin(
            4 * np.pi * (h - 18) / 24
        )
    elif class_name == "com":  # تجاری: پیک صبح-ظهر
        prof = 0.55 + 0.30 * np.sin(2 * np.pi * (h - 11) / 24)
    else:  # صنعتی: نسبتاً تخت
        prof = 0.75 + 0.05 * np.sin(2 * np.pi * (h - 9) / 24)

    prof = np.clip(prof, 0.15, None)
    return prof / prof.mean()


def simulate_customers(idx, n_customers=400, seed=7):
    rng = np.random.default_rng(seed)

    classes = rng.choice(["res", "com", "ind"], size=n_customers, p=[0.75, 0.20, 0.05])

    A = np.where(
        classes == "res",
        rng.uniform(0.3, 2.0, n_customers),
        np.where(
            classes == "com",
            rng.uniform(1.0, 8.0, n_customers),
            rng.uniform(5.0, 20.0, n_customers),
        ),
    )

    prof_res = base_profile("res")
    prof_com = base_profile("com")
    prof_ind = base_profile("ind")

    hour = idx.hour.values
    dow = idx.dayofweek.values

    week_factor = np.ones(len(idx))
    weekend = (dow >= 5)
    week_factor[weekend] = 0.92

    com_factor = np.ones(len(idx))
    com_factor[weekend] = 0.70

    P = np.zeros((len(idx), n_customers), dtype=float)
    for i in range(n_customers):
        if classes[i] == "res":
            base = prof_res[hour]
            factor = week_factor
        elif classes[i] == "com":
            base = prof_com[hour]
            factor = com_factor
        else:
            base = prof_ind[hour]
            factor = np.ones(len(idx))

        noise = rng.normal(0, 0.08 * A[i], size=len(idx))
        events = (rng.random(len(idx)) < 0.003) * rng.lognormal(
            mean=0.0, sigma=0.6, size=len(idx)
        ) * (0.3 * A[i])

        P[:, i] = A[i] * base * factor + noise + events

    return classes, A, np.clip(P, 0.0, None)


def apply_temperature_effect(P, temp, alpha=0.03, theta_cool=30.0):
    g = 1 + alpha * np.maximum(0.0, temp - theta_cool)
    return P * g[:, None]


# ------------------------------------------
# 2) صورتحساب/اندازه‌گیری + Missing
# ------------------------------------------
def inject_ntl(P_true_kw, theft_rate=0.12, gamma_max=0.35, seed=99):
    rng = np.random.default_rng(seed)
    n = P_true_kw.shape[1]
    is_theft = rng.random(n) < theft_rate
    gamma = np.zeros(n)
    gamma[is_theft] = rng.uniform(0.05, gamma_max, size=is_theft.sum())
    P_billed = P_true_kw * (1 - gamma[None, :])
    return P_billed, is_theft, gamma


def add_meter_noise_and_bias(P_billed_kw, bias_std=0.02, noise_std=0.03, seed=11):
    rng = np.random.default_rng(seed)
    n = P_billed_kw.shape[1]
    bias = rng.normal(0, bias_std, size=n)
    noise = rng.normal(0, noise_std, size=P_billed_kw.shape)
    P_meas = (1 + bias[None, :]) * P_billed_kw * (1 + noise)
    return np.clip(P_meas, 0, None)


def make_missing(P_meas, miss_rate=0.08, burst_prob=0.01, burst_len=12, seed=21):
    rng = np.random.default_rng(seed)
    X = P_meas.copy()

    # missing تصادفی
    mask_rand = rng.random(X.shape) < miss_rate
    X[mask_rand] = np.nan

    # missing پشت‌سرهم (قطعی)
    for j in range(X.shape[1]):
        if rng.random() < 0.35:
            for t in range(X.shape[0]):
                if rng.random() < burst_prob:
                    end = min(X.shape[0], t + burst_len)
                    X[t:end, j] = np.nan
    return X


# ------------------------------------------
# 3) ایمپیوت: Baseline و مدل‌های رگرسیون
# ------------------------------------------
def impute_hourly_mean_single(series, idx):
    s = pd.Series(series, index=idx)
    grp = s.groupby(idx.hour).transform("mean")
    s2 = s.fillna(grp)
    s2 = s2.fillna(s2.mean())
    return s2.values


def build_supervised_features(s_series, temp, idx, lags=(1, 24)):
    """
    ویژگی‌ها: hour, dow, temp, lag_1, lag_24
    خروجی: X, y برای نقاطی که NaN نیستند و lagها موجودند
    """
    s = pd.Series(s_series, index=idx)

    feats = pd.DataFrame(index=idx)
    feats["hour"] = idx.hour
    feats["dow"] = idx.dayofweek
    feats["temp"] = temp

    for L in lags:
        feats[f"lag_{L}"] = s.shift(L)

    data = pd.concat([feats, s.rename("y")], axis=1).dropna()
    X = data.drop(columns=["y"])
    y = data["y"]
    return X, y


def model_impute_single_customer(series_with_nan, temp, idx, model, lags=(1, 24)):
    """
    مدل را روی نقاط غیر-NaN آموزش می‌دهد، Missingها را پیش‌بینی می‌کند،
    باقی NaNها را با HourlyMean پر می‌کند.
    """
    s = pd.Series(series_with_nan, index=idx)

    X_train, y_train = build_supervised_features(s.values, temp, idx, lags=lags)
    if len(X_train) < 100:
        # اگر دیتا کم بود، fallback
        return impute_hourly_mean_single(s.values, idx)

    model.fit(X_train, y_train)

    # ساخت ویژگی‌ها برای همه زمان‌ها (برای پیش‌بینی نقاط missing)
    feats = pd.DataFrame(index=idx)
    feats["hour"] = idx.hour
    feats["dow"] = idx.dayofweek
    feats["temp"] = temp
    feats["lag_1"] = s.shift(1)
    feats["lag_24"] = s.shift(24)

    miss_idx = s.index[s.isna()]
    X_pred = feats.loc[miss_idx].dropna()
    if len(X_pred) > 0:
        pred = model.predict(X_pred)
        s.loc[X_pred.index] = pred

    # باقی‌مانده را با روش آماری پر می‌کنیم
    s_filled = pd.Series(impute_hourly_mean_single(s.values, idx), index=idx)
    return s_filled.values


# ------------------------------------------
# 4) KDE + PDF (هیستوگرام چگالی) بدون SciPy
# ------------------------------------------
def kde_density_1d(x, grid, bandwidth=None):
    """
    KDE ساده با KernelDensity از sklearn (بدون نیاز به SciPy)
    """
    from sklearn.neighbors import KernelDensity

    x = np.asarray(x, dtype=float)
    x = x[np.isfinite(x)]
    if len(x) < 30:
        return np.zeros_like(grid, dtype=float)

    # bandwidth: یک مقدار ساده و پایدار
    if bandwidth is None:
        std = np.std(x) + 1e-9
        bandwidth = 1.06 * std * (len(x) ** (-1 / 5))  # Silverman-ish
        bandwidth = max(bandwidth, 1e-3)

    kde = KernelDensity(kernel="gaussian", bandwidth=bandwidth)
    kde.fit(x.reshape(-1, 1))

    log_d = kde.score_samples(grid.reshape(-1, 1))
    return np.exp(log_d)


def plot_pdf_kde(errors, title, out_png, pdf_pages=None, bins=60):
    """
    PDF: هیستوگرام چگالی
    KDE: منحنی KDE
    """
    e = np.asarray(errors, dtype=float)
    e = e[np.isfinite(e)]
    if len(e) == 0:
        return

    lo, hi = np.percentile(e, [1, 99])
    # اگر خیلی کم بود، کمی گسترش
    if abs(hi - lo) < 1e-6:
        lo, hi = lo - 1, hi + 1

    grid = np.linspace(lo, hi, 400)
    kde = kde_density_1d(e, grid)

    plt.figure()
    plt.hist(e, bins=bins, density=True, alpha=0.5, label="PDF (hist density)")
    plt.plot(grid, kde, linewidth=2, label="KDE")
    plt.title(title)
    plt.xlabel("Error")
    plt.ylabel("Density")
    plt.legend()
    plt.tight_layout()
    plt.savefig(out_png, dpi=200)
    if pdf_pages is not None:
        pdf_pages.savefig()
    plt.close()


def plot_box_abs_error(abs_errors, title, out_png, pdf_pages=None):
    """
    Boxplot خطای مطلق — برای هر مدل جدا
    """
    a = np.asarray(abs_errors, dtype=float)
    a = a[np.isfinite(a)]
    if len(a) == 0:
        return

    plt.figure()
    plt.boxplot(a, vert=True, showfliers=True)
    plt.title(title)
    plt.ylabel("Absolute Error")
    plt.xticks([1], ["|error|"])
    plt.tight_layout()
    plt.savefig(out_png, dpi=200)
    if pdf_pages is not None:
        pdf_pages.savefig()
    plt.close()


def plot_box_compare(abs_err_dict, out_png, pdf_pages=None):
    """
    Boxplot مقایسه‌ای همه مدل‌ها کنار هم (اضافی ولی مفید)
    """
    names = list(abs_err_dict.keys())
    data = []
    for k in names:
        a = np.asarray(abs_err_dict[k], dtype=float)
        a = a[np.isfinite(a)]
        data.append(a)

    plt.figure(figsize=(max(8, 1.5 * len(names)), 5))
    plt.boxplot(data, labels=names, vert=True, showfliers=True)
    plt.title("Absolute Error Comparison (All Models)")
    plt.ylabel("Absolute Error")
    plt.xticks(rotation=20)
    plt.tight_layout()
    plt.savefig(out_png, dpi=200)
    if pdf_pages is not None:
        pdf_pages.savefig()
    plt.close()


# ------------------------------------------
# 5) اجرای اصلی: چند مدل + مقایسه + PDF/KDE/Box
# ------------------------------------------
def main(
    days=30,
    n_customers=400,
    eval_customers=120,   # برای سرعت/مقایسه (می‌تونی ببری بالا)
    start="2025-08-01",
):
    idx = make_time_index(days=days, start=start)
    temp = make_temperature(idx)

    classes, A, P = simulate_customers(idx, n_customers=n_customers)
    P_true = apply_temperature_effect(P, temp)

    # داده اندازه‌گیری‌شده (صورتحساب + نویز)
    P_billed, is_theft, gamma = inject_ntl(P_true, theft_rate=0.12)
    P_meas = add_meter_noise_and_bias(P_billed)

    # Missing
    P_miss = make_missing(P_meas, miss_rate=0.08)

    # انتخاب زیرمجموعه مشتری‌ها برای مقایسه مدل‌ها (همه مدل‌ها روی یک subset یکسان)
    rng = np.random.default_rng(202)
    cust_idx = rng.choice(np.arange(n_customers), size=min(eval_customers, n_customers), replace=False)

    # مدل‌ها
    models = {
        "HourlyMean": None,
        "Ridge": Ridge(alpha=1.0, random_state=0),
        "KNN": KNeighborsRegressor(n_neighbors=15, weights="distance"),
        "RandomForest": RandomForestRegressor(
            n_estimators=250, random_state=0, n_jobs=-1, max_depth=None
        ),
        "HistGB": HistGradientBoostingRegressor(
            max_depth=6, learning_rate=0.07, random_state=0
        ),
    }

    # برای ذخیره خطاهای هر مدل
    signed_err = {}   # error = pred - true
    abs_err = {}      # |error|
    metrics = []      # MAE/RMSE

    # فایل PDF چند صفحه‌ای
    with PdfPages("chapter4_error_plots.pdf") as pdf:
        for name, model in models.items():
            errs = []
            abses = []

            for j in cust_idx:
                x_nan = P_miss[:, j]      # سری دارای Missing
                y_true = P_meas[:, j]     # “حقیقت” برای ارزیابی (قبل از حذف شدن)

                miss_mask = np.isnan(x_nan)
                if miss_mask.sum() == 0:
                    continue

                if name == "HourlyMean":
                    rec = impute_hourly_mean_single(x_nan, idx)
                else:
                    rec = model_impute_single_customer(x_nan, temp, idx, model)

                e = (rec[miss_mask] - y_true[miss_mask])
                errs.append(e)
                abses.append(np.abs(e))

            if len(errs) == 0:
                continue

            errs = np.concatenate(errs)
            abses = np.concatenate(abses)

            signed_err[name] = errs
            abs_err[name] = abses

            # MAE / RMSE روی نقاط Missing (فقط همان جاهایی که تخمین زدیم)
            mae = float(np.mean(np.abs(errs)))
            rmse = float(np.sqrt(np.mean(errs**2)))
            metrics.append({"model": name, "MAE_missing": mae, "RMSE_missing": rmse})

            # --- رسم PDF+KDE خطاهای علامت‌دار
            plot_pdf_kde(
                errors=errs,
                title=f"{name} - Signed Error Distribution (PDF+KDE)",
                out_png=f"fig_err_pdf_kde_signed_{name}.png",
                pdf_pages=pdf,
            )

            # --- رسم PDF+KDE خطاهای مطلق
            plot_pdf_kde(
                errors=abses,
                title=f"{name} - Absolute Error Distribution (PDF+KDE)",
                out_png=f"fig_err_pdf_kde_abs_{name}.png",
                pdf_pages=pdf,
            )

            # --- Boxplot جداگانه برای هر مدل (خطای مطلق)
            plot_box_abs_error(
                abs_errors=abses,
                title=f"{name} - Absolute Error Boxplot",
                out_png=f"fig_box_abs_error_{name}.png",
                pdf_pages=pdf,
            )

        # یک Boxplot مقایسه‌ای هم اضافه می‌کنیم (همه مدل‌ها کنار هم)
        if len(abs_err) > 0:
            plot_box_compare(
                abs_err_dict=abs_err,
                out_png="fig_box_abs_error_compare_all_models.png",
                pdf_pages=pdf,
            )

    # جدول مقایسه
    metrics_df = pd.DataFrame(metrics).sort_values(by="RMSE_missing")
    metrics_df.to_csv("chapter4_imputation_models_comparison.csv", index=False, encoding="utf-8-sig")

    print("=== Model Comparison on Missing Points ===")
    print(metrics_df)

    print("\nSaved outputs:")
    print("- chapter4_imputation_models_comparison.csv")
    print("- chapter4_error_plots.pdf  (multi-page)")
    print("- fig_err_pdf_kde_signed_<MODEL>.png")
    print("- fig_err_pdf_kde_abs_<MODEL>.png")
    print("- fig_box_abs_error_<MODEL>.png")
    print("- fig_box_abs_error_compare_all_models.png")


if __name__ == "__main__":
    main()


/tmp/ipykernel_17/1340642952.py:300: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.boxplot(data, labels=names, vert=True, showfliers=True)


=== Model Comparison on Missing Points ===
          model  MAE_missing  RMSE_missing
1         Ridge     0.368353      0.693983
0    HourlyMean     0.397411      0.755170
2           KNN     0.406238      0.767997
3  RandomForest     0.427908      0.802811
4        HistGB     0.430072      0.809642

Saved outputs:
- chapter4_imputation_models_comparison.csv
- chapter4_error_plots.pdf  (multi-page)
- fig_err_pdf_kde_signed_<MODEL>.png
- fig_err_pdf_kde_abs_<MODEL>.png
- fig_box_abs_error_<MODEL>.png
- fig_box_abs_error_compare_all_models.png
